# Broker Agent Experimental Code
This is a notebook for exploring the codebase and testing ideas.
Goal is the make sure we can connect to a website and scrape/browse/interact with the page.

In [19]:
from datetime import datetime
import re
import asyncio
import traceback
import chromadb

from langchain_ollama import ChatOllama
from playwright.async_api import async_playwright

from broker_agent.config.settings import BrokerAgentConfig

In [20]:
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

from database.alembic.models.models import Apartment
from sqlalchemy.exc import IntegrityError

In [21]:
config = BrokerAgentConfig.from_yaml_and_env()

In [22]:
ollama = ChatOllama(base_url=config.OLLAMA_BASE_URL, model=config.llm)

In [23]:
async def extract_playwright_script(script_content):
    """
    Extract the playwright script from between the ```playwright and ``` markers

    Args:
        script_content (str): The content containing the playwright script

    Returns:
        str or None: The extracted script if found, None otherwise
    """
    playwright_code = re.search(r"```playwright\n(.*?)```", script_content, re.DOTALL)
    if playwright_code:
        return playwright_code.group(1)
    return None

In [24]:
# TODO: May need to filter a11y tree to ensure best model understanding
def format_a11y_tree(tree_data):
    """Format an accessibility tree for better readability."""
    
    # Handle input that's already a dictionary
    if isinstance(tree_data, dict):
        tree = tree_data
    else:
        raise TypeError("Input must be a dictionary or string")
    
    # Format the tree recursively
    def format_node(node, depth=0):
        indent = "  " * depth
        result = []
        
        # Node header with role and name
        header = f"{indent}- {node['role']}: \"{node['name']}\""
        result.append(header)
        
        # Process children if present
        if 'children' in node and node['children']:
            if len(node['children']) > 10:
                # Summarize if there are many children
                shown_children = node['children'][:5]
                result.append(f"{indent}  ↳ Children ({len(node['children'])} total):")
                for child in shown_children:
                    result.extend(format_node(child, depth + 2))
                result.append(f"{indent}  ↳ ... {len(node['children']) - 5} more children ...")
            else:
                result.append(f"{indent}  ↳ Children:")
                for child in node['children']:
                    result.extend(format_node(child, depth + 2))
        
        return result
    
    formatted = format_node(tree)
    return "\n".join(["# Accessibility Tree Structure", "```", *formatted, "```"])

In [25]:
async def get_test_playwright_script() -> str:
    return """```playwright
async def main():
    # Get elements using aria-labels and roles
    price_button = page.get_by_role("button", name="Price")
    bedrooms_select = page.get_by_label('Beds / Baths')
    no_fee_checkbox = page.get_by_label('No Fee Only')

    # Click the Price button to open price filter
    await price_button.click()

    # Click the no fee checkbox since its under the pricing menu
    no_fee_checkbox = page.get_by_label('No Fee Only')
    await no_fee_checkbox.click()

    # Wait for and fill in min/max price fields
    await page.wait_for_selector("input[placeholder='No min']")
    min_price_input = page.get_by_placeholder("No min")
    await min_price_input.click()
    await min_price_input.fill("3000")

    max_price_input = page.get_by_placeholder("Max")
    await max_price_input.click()
    await max_price_input.fill("4000")
    
    # Press ESC to dismiss dropdown options that might be blocking the Done button
    await page.keyboard.press("Escape")

    # Select Studio from Bedrooms dropdown
    await bedrooms_select.click()
    studio_option = page.get_by_test_id("desktop-filter").get_by_text("Studio")
    await studio_option.click()

    # Press ESC to dismiss dropdown options that might be blocking the Done button
    await page.keyboard.press("Escape")
```
"""


In [26]:
async def get_test_save_listings_playwright_script() -> str:
    return """```playwright
async def main():
    links = set()
    max_depth = 2
    i = 0

    while i < max_depth:
        # Extract all listing details containers
        listing_containers = await page.query_selector_all('.ListingCard-module__listingDetailsContainer')

        # Extract href from each listing
        for container in listing_containers:
            a_element = container.query_selector('a[href]')
            if a_element:
                href = await a_element.get_attribute('href')
                links.add(href)
            
        anchor_tags = await page.query_selector_all('a.ListingDescription-module__addressTextAction___xAFZJ[href]')
        for anchor in anchor_tags:
            href = await anchor.get_attribute('href')
            links.add(href)

        # Check if pagination exists using aria-label
        pagination = page.get_by_role('region', name='Pagination')

        if not pagination:
            break

        # Get next button by label
        next_button = page.get_by_label("Next Page")

        # Click the next button to load more listings
        await next_button.click()

        # Wait for content to load (adjust timing as needed)
        await page.wait_for_event("load", timeout=60000)

        # TODO: Implement exponential backoff for politeness when visiting
        await asyncio.sleep(10)
        i += 1
    return list(links)
```
"""


In [27]:
# TODO: Do we need a all the DOM content to generate a script?
# The following is the DOM content and accessibility tree of the website:
# <dom_content>
# {dom_content}
# </dom_content>

async def generate_search_playwright_script(website: str, dom: str, a11y_tree: str, error_message: str) -> str:
    generate_playwright_script_prompt = """
    Generate a Python Playwright script to search for a studio apartment
    in the $3000-$4000 range. Note the following playwright code has already been executed:

    <already_executed_code>
    await page.goto(website)
    dom_content = await page.content()
    a11y_tree = await page.accessibility.snapshot
    </already_executed_code>

    <aria_labels>
    - 'Price'
    - 'Beds / Baths
    - 'No min'
    - 'No max'
    - 'Options' (This one is a form for the min/max price in dollars)
    - 'No Fee Only'
    - 'Done' (This one sets the price level)
    - 'Search' (This one clicks the search button after setting the price and number of bedrooms)
    </aria_labels>
    
    <rules>
    - Start the script by async grabbing elements with `page.get_by_role()` or `page.get_by_label()` or `page.get_by_text()` or similar functions since we already
    have the DOM content on the current page
        - Don't use `await` when calling these functions, just call them sync i.e `page.get_by_role("button", name="Price")` or `page.get_by_label("...")`
    - You also don't need to wait for the page to load i.e `page.wait_for_event("load")` since
    the page is already loaded
    - Click the 'No Fee Only' button before filling in the below input forms with min/max price
    - Use `await page.wait_for_selector("input[placeholder='No min']")` and `await page.wait_for_selector("input[placeholder='No max']")` to select the minimum and maximum price boxes
        - Then click them like `await page.get_by_placeholder("No min").click()`
        - Then enter fields i.e `await page.get_by_placeholder("No min").fill("3000")`
        - Finally, after entering the price, press escape to exit the price selection menu `await page.keyboard.press("Escape")`
    - Make sure to click elements like form fields before entering data into them
    - Use the `aria-label` attributes from the a11y tree to find the elements to interact with
    - The search script should handle clicking the 'Price' button, filling out the min and max price, 
    filling out the 'Bedrooms' dropdown and selecting 'Studio', and clicking the 'Search' button.
    - Make sure that the generated playwright code uses the async playwright API i.e (from playwright.async_api import async_playwright).
    - Don't run the code directly with asyncio.run() - just return the code in the main() function.
    - You don't need to use the page.goto() function since we are already on the target website.
    - Use the following test ID filter to filter by apartment type: `page.get_by_test_id("desktop-filter").get_by_text("Studio")`
    </rules>

    If there was an error in the previous playwright script, please use the following error message to generate a new playwright script:
    <error_message>
    {error_message}
    </error_message>

    <a11y_tree>
    {a11y_tree}
    </a11y_tree>

    Return the playwright script in markdown format like the following:
```playwright
async def main():
    # Get elements using aria-labels and roles
    price_button = page.get_by_role("button", name="Price")
    bedrooms_select = page.get_by_label('Beds / Baths')
    no_fee_checkbox = page.get_by_label('No Fee Only')

    # Click the Price button to open price filter
    await price_button.click()

    # Click the no fee checkbox since its under the pricing menu
    no_fee_checkbox = page.get_by_label('No Fee Only')
    await no_fee_checkbox.click()

    # Wait for and fill in min/max price fields
    await page.wait_for_selector("input[placeholder='No min']")
    min_price_input = page.get_by_placeholder("No min")
    await min_price_input.click()
    await min_price_input.fill("3000")

    max_price_input = page.get_by_placeholder("Max")
    await max_price_input.click()
    await max_price_input.fill("4000")
    
    # Press ESC to dismiss dropdown options that might be blocking the Done button
    await page.keyboard.press("Escape")

    # Select Studio from Bedrooms dropdown
    await bedrooms_select.click()
    studio_option = page.get_by_test_id("desktop-filter").get_by_text("Studio")
    await studio_option.click()

    # Press ESC to dismiss dropdown options that might be blocking the Done button
    await page.keyboard.press("Escape")
    ...
```

    Playwright script:
    """
    filled_prompt = generate_playwright_script_prompt.format(a11y_tree=format_a11y_tree(a11y_tree), error_message=error_message)
    print(f"Prompt = {filled_prompt}")
    return (await ollama.ainvoke(filled_prompt)).content

In [28]:
async def generate_save_listings_playwright_script(website: str, dom: str, a11y_tree: str, error_message: str) -> str:
    """
    Generate a playwright script to extract and save all apartment listings after having searched and filtered on streeteasy
    """
    generate_playwright_script_prompt = """
    Generate a Python Playwright scrape all apartment listings on this page. Grab all the listings of the following form that you can see,
    then scroll down to see more listings and grab those. Continue to scroll until you see an element with `aria-label="Pagination"` where you stop
    and then press the next arrow button with `aria-labelledby="next-arrow-label"`

    Extract the data from anchor tags in the page of the following form:
    <apartment_listing_html>
    <a class="text-action_baseTextAction_QUkYk text-action_bodyStyles_VRnQz text-action_brandColor_twufp ListingDescription-module__addressTextAction___xAFZJ" color="#1876DB" href="https://streeteasy.com/..." target="_blank">Apartment Description</a>
    </apartment_listing_html>

    Extract the following elements:
    <elements_to_extract>
    - link to listing (href)
    </elements_to_extract>
    
    <rules>
    - Extract ONLY the link (href) to the listing, not any other information
    - Include all the code in the `async def main()` function
    - main() should take no arguments and return a list[str] where each str is a link to a listing
    - Start the script by async grabbing elements with `page.get_by_role()` or `page.get_by_label()` or `page.get_by_text()` or similar functions since we already
    have the DOM content on the current page
        - Don't use `await` when calling these functions, just call them sync i.e `page.get_by_role("button", name="Price")` or `page.get_by_label("...")`
    - You also don't need to wait for the page to load i.e `page.wait_for_event("load")` since the page is already loaded
    - Use the `aria-label` attributes from the a11y tree to find the elements to interact with
    - Make sure that the generated playwright code uses the async playwright API i.e (from playwright.async_api import async_playwright).
    - Don't run the code directly with asyncio.run() - just return the code in the main() function.
    - You don't need to use the page.goto() function since we are already on the target website.
    - Use the 'ListingCard-module__listingDetailsContainer' class to find the information for each listing
    - Use the <ul> tag to look for where the listings are in the DOM
    </rules>

    If there was an error in the previous playwright script, please use the following error message to generate a new playwright script:
    <error_message>
    {error_message}
    </error_message>

    <a11y_tree>
    {a11y_tree}
    </a11y_tree>

    Generate a playwright script in markdown format like the following:
    <playwright_script>
```playwright
async def main():
    links = set()

    while True:
        # Extract all listing details containers
        listing_containers = await page.query_selector_all('.ListingCard-module__listingDetailsContainer')

        # Extract href from each listing
        for container in listing_containers:
            a_element = await container.query_selector('a[href]')
            if a_element:
                href = await a_element.get_attribute('href')
                links.add(href)

        # Check if pagination exists using aria-label
        pagination = await page.get_by_role('region', name='Pagination')

        if not pagination:
            break

        # Get next button by label
        next_button = await page.get_by_label('button', 'next-arrow-label')

        # Click the next button to load more listings
        await next_button.click()

        # Wait for content to load (adjust timing as needed)
        await page.wait_for_event("load")

    return list(links)
```
    </playwright_script>

    The generated playwright script should loop scrolling the page until it reaches the `next-arrow-label` whereupon it clicks the `next-arrow-label`
    and repeats until the `next-arrow-label` is not found. The script should then break and return all the found links as a list[str].

    Playwright script:
    """
    filled_prompt = generate_playwright_script_prompt.format(a11y_tree=format_a11y_tree(a11y_tree), error_message=error_message)
    print(f"Prompt = {filled_prompt}")
    return (await ollama.ainvoke(filled_prompt)).content

In [29]:
async def generate_test_scrape_listing_playwright_script(website: str, dom: str, a11y_tree: str, error_message: str):
    generate_test_scrape_listing_playwright_script_prompt = """
    Generate a playwright script to scrape the details of an apartment listing.
    Use the below instructions and a11y tree to generate the playwright script.

    <rules>
    - press the data-testid="next-image-button until you get all the image urls on the page (maximum 20)
    </rules>

    <information_to_extract>
    - name of the apartment (data-testid="homeAddress")
    - price of the apartment (data-testid="priceInfo")
    - description of the apartment (data-testid="about-section")
    - available date of the apartment (data-testid="rentalListingSpec-available")
    - days on market of the apartment (data-testid="rentalListingSpec-daysOnMarket")
    - image urls of the apartment
        - Use the `data-testid="next-image-button"` component to tab between the image urls in a for loop
    </information_to_extract>

    <rules>
    - Extract ONLY the link (href) to the listing, not any other information
    - Include all the code in the `async def main()` function
    - main() should take no arguments and return a list[str] where each str is a link to a listing
    - Start the script by async grabbing elements with `page.get_by_role()` or `page.get_by_label()` or `page.get_by_text()` or similar functions since we already
    have the DOM content on the current page
        - Don't use `await` when calling these functions, just call them sync i.e `page.get_by_role("button", name="Price")` or `page.get_by_label("...")`
    - You also don't need to wait for the page to load i.e `page.wait_for_event("load")` since the page is already loaded
    - Use the `aria-label` or `data-testid` attributes from the a11y tree to find the elements to interact with
    - Make sure that the generated playwright code uses the async playwright API i.e (from playwright.async_api import async_playwright).
    - Don't run the code directly with asyncio.run() - just return the code in the main() function.
    - You don't need to use the page.goto() function since we are already on the target website.
    </rules>

    <a11y_tree>
    {a11y_tree}
    </a11y_tree>

    Generate a playwright script in markdown format like the following:
    <playwright_script>
```playwright
async def main(page: Page) -> list[dict[str, any]]:
    # Extract textual information
    name = (await page.locator('[data-testid="homeAddress"]').text_content(timeout=5000) or "").strip()
    price = (await page.locator('[data-testid="priceInfo"]').text_content(timeout=5000) or "").strip()
    description = (await page.locator('[data-testid="about-section"]').text_content(timeout=5000) or "").strip()
    available_date = (await page.locator('[data-testid="rentalListingSpec-available"]').text_content(timeout=5000) or "").strip()
    days_on_market = (await page.locator('[data-testid="rentalListingSpec-daysOnMarket"]').text_content(timeout=5000) or "").strip()

    apartment_data = {{
        "name": name,
        "price": price,
        "description": description,
        "available_date": available_date,
        "days_on_market": days_on_market,
        "image_urls": image_urls,
    }}

    return [apartment_data]
```
    </playwright_script>

    The script should extract all the information_to_extract and return it in a list(dict) of the following form:
    <expected_output>
    [
        {{
            "name": "Apartment Name",
            "price": "Price",
            "description": "Description",
            "available_date": "Available Date",
            "days_on_market": "Days on Market",
            "image_urls": ["Image URL 1", "Image URL 2", "Image URL 3"]
        }}
    ]
    </expected_output>
    
    Playwright script:
"""

    filled_prompt = generate_test_scrape_listing_playwright_script_prompt.format(a11y_tree=format_a11y_tree(a11y_tree))
    print(f"Prompt = {filled_prompt}")
    return (await ollama.ainvoke(filled_prompt)).content

In [30]:
playwright_image_url_extractor_prompt = """
    # Image extraction
    image_urls: list[str] = []
    seen_srcs: set[str] = set()
    next_button_locator = page.locator('[data-testid="next-image-button"]')

    # This list of selectors attempts to find the main gallery image.
    # It's a guess and might need adjustment based on the actual page structure.
    # Order matters: more specific or reliable selectors should come first.
    main_image_candidate_selectors = [
        'img[data-testid="gallery-main-image"][src]',
        'div[aria-roledescription="carousel"] img[src]:visible',
        'ul[role="listbox"] li[aria-selected="true"] img[src]',
        'figure[data-testid*="gallery"] img[src]',
        'img[data-testid*="image-slide"][src]:visible',
        'section[aria-label*="gallery"] img[src]:visible',
        'img[alt*="Photo of listing" i][src]:visible',
        'img[alt*="Slide" i][src]:visible',
        'div[class*="gallery"] img[src]:visible',
        'div[class*="carousel"] img[src]:visible',
        'article img[src]:visible', # More generic fallbacks
        'main img[src]:visible',
    ]

    current_image_element = None # To store the locator of the found image for comparison

    for i in range(20):  # Max 20 images
        current_src = None
        
        # Attempt to find the current image's src using candidate selectors
        for selector_index, sel in enumerate(main_image_candidate_selectors):
            try:
                candidate_element = page.locator(sel).first
                # Wait for element to be attached, then check visibility with short timeout
                await candidate_element.wait_for(state='attached', timeout=750)
                if await candidate_element.is_visible(timeout=750):
                    src_val = await candidate_element.get_attribute('src')
                    if src_val and src_val.startswith('http'):
                        current_src = src_val
                        current_image_element = candidate_element # Store the reference
                        # print(f"Debug: Image found with selector '{sel}': {current_src}")
                        break 
            except Exception:
                # print(f"Debug: Selector '{sel}' failed or timed out.")
                continue
        
        # If no image found via specific selectors, try a very general one as a last resort for the first image.
        if not current_src and i == 0: # Only for the very first image attempt
            try:
                general_img_loc = page.locator('img[src]:visible').first
                await general_img_loc.wait_for(state='attached', timeout=750)
                if await general_img_loc.is_visible(timeout=750):
                    src_val = await general_img_loc.get_attribute('src')
                    if src_val and src_val.startswith('http'):
                        current_src = src_val
                        current_image_element = general_img_loc
                        # print(f"Debug: Image found with general selector: {current_src}")
            except Exception:
                # print("Debug: General 'img[src]:visible' selector failed for first image.")
                pass


        if current_src:
            if current_src not in seen_srcs:
                image_urls.append(current_src)
                seen_srcs.add(current_src)
            else:
                # If it's not the first image collected and we've seen this src, we've likely cycled.
                if len(image_urls) > 0:
                    print(f"Info: Duplicate image src ({current_src}) encountered. Assuming end of unique images.")
                    break
        else:
            # If no image src could be found (especially after the first iteration), stop.
            if i > 0 : 
                print("Info: Could not determine current image src in this iteration. Stopping image collection.")
                break
            # If it's the first iteration and no image found, it's a problem with initial selectors or page structure.
            # print("Warning: Could not determine current image src on the first attempt.")


        # Check if the next button is available and clickable
        if await next_button_locator.is_visible(timeout=1000) and await next_button_locator.is_enabled(timeout=1000):
            prev_src_for_wait = current_src # Store src before click to wait for change

            await next_button_locator.click()

            # Wait for the image to change. This is more robust than a fixed timeout.
            if current_image_element and prev_src_for_wait:
                try:
                    # Expect the 'src' attribute of the *same element reference* to change.
                    # This works if the element itself is stable and only its src updates.
                    await expect(current_image_element).not_to_have_attribute('src', prev_src_for_wait, timeout=5000)
                except AssertionError: # This means src did not change within timeout
                     print(f"Info: Image src '{prev_src_for_wait}' did not change after click, or element reference is stale. Trying generic wait.")
                     await page.wait_for_timeout(1000) # Fallback wait
                except Exception: # Other errors like element disappearing
                    # print(f"Debug: Error waiting for image src to change. Element might have been replaced.")
                    await page.wait_for_timeout(1500) # Fallback: generic wait if element changed
            else:
                # Fallback to generic timeout if no stable image element or src to compare against
                await page.wait_for_timeout(1500)
        else:
            print("Info: Next button not available or not clickable. Stopping image collection.")
            break
"""

In [31]:
async def execute_llm_script(script_code, context_objects):
    # Define a local namespace with your objects
    namespace = context_objects.copy()
    
    # Execute the code in this namespace
    exec(script_code, namespace)
    
    # If the script defines a main function, call it and return its value
    if 'main' in namespace and callable(namespace['main']):
        result = await namespace['main']()
        return result
    else:
        raise ValueError("No main function found in the script")

## Establish DB Connection
This codebase uses a PostgreSQL database to store the data. The following code establishes a connection to the database.

In [32]:
load_dotenv()

# Construct the database URL from environment variables or use defaults
db_user = os.environ.get("POSTGRES_USER", "postgres")
db_password = os.environ.get("POSTGRES_PASSWORD", "postgres")
db_host = os.environ.get("POSTGRES_HOST", "localhost")
db_port = os.environ.get("POSTGRES_PORT", "5432")
db_name = os.environ.get("POSTGRES_DB", "postgres")

# Create the database URL
db_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

# Create the SQLAlchemy engine
engine = create_engine(db_url)

# Create a sessionmaker
Session = sessionmaker(bind=engine)

# Create a session
session = Session()

# Test the connection
try:
    connection = engine.connect()
    print("Successfully connected to the PostgreSQL database!")
    connection.close()
except Exception as e:
    print(f"Error connecting to the database: {e}")


Successfully connected to the PostgreSQL database!


## Python vs. JS Playwright
If there's too few examples for python playwright, then try out the JS playwright instead

In [33]:
async def extract_apartment_details(page, listing_url):
    """
    Navigate to a listing URL and extract apartment details using an LLM.
    
    Args:
        page: Playwright page object
        listing_url: URL of the apartment listing
    
    Returns:
        dict: Apartment details with the following keys:
            - name (str): Name of the apartment
            - description (str): Description of the apartment
            - available_date (datetime): Date when the apartment is available
            - days_on_market (int): Number of days the listing has been on the market
            - image_urls (list): List of image URLs for the apartment
    """
    try:
        await page.goto(listing_url, timeout=60000)
        dom_content = await page.content()
        
        # Here you would make an LLM call to extract the information
        # The prompt would be customized to extract structured data from the listing page
        
        # Placeholder for LLM call
        # llm_response = await call_llm_with_prompt(prompt="")
        
        # For now, return a dummy structured response
        # In a real implementation, parse the LLM response into this structure
        return {
            "name": "",
            "description": "",
            "available_date": datetime.now(),
            "days_on_market": 0,
            "image_urls": []
        }
    except Exception as e:
        print(f"Error extracting details from {listing_url}: {e}")
        return None
    
async def save_listings_to_db(listings: list[dict[str, any]], Session):
    """
    Save apartment listings to the database.
    
    Args:
        listings: List of listing details to save
        Session: SQLAlchemy sessionmaker
    """
    # Use a with block to manage the session lifecycle
    with Session() as db_session:
        for listing in listings:
            try:
                # Check if the apartment already exists
                existing_apartment = db_session.query(Apartment).filter_by(link=listing["link"]).first()
                if not existing_apartment:
                    # Extract days on market as an integer
                    days_on_market = 0
                    try:
                        if isinstance(listing["days_on_market"], str):
                            # Extract numeric value from string like "Days on market50 days"
                            days_match = re.search(r'(\d+)', listing["days_on_market"])
                            if days_match:
                                days_on_market = int(days_match.group(1))
                    except Exception as e:
                        print(f"Warning: Could not parse days_on_market from {listing['days_on_market']}: {e}")
                    
                    # Parse available date
                    available_date = datetime.now()
                    try:
                        if isinstance(listing["available_date"], str):
                            if "Available now" in listing["available_date"]:
                                available_date = datetime.now()
                            elif isinstance(listing["available_date"], str):
                                # Try to extract date if in standard format
                                available_date = datetime.strptime(listing["available_date"], "%Y-%m-%d")
                    except Exception as e:
                        print(f"Warning: Could not parse available_date from {listing['available_date']}: {e}")
                    
                    # Parse price - extract numeric value from string like "$3,146For RentPrice Drop$51No fee"
                    price = 0
                    try:
                        if isinstance(listing["price"], str):
                            # Extract the first price value (e.g., $3,146)
                            price_match = re.search(r'\$([0-9,]+)', listing["price"])
                            if price_match:
                                # Remove commas and convert to float
                                price = float(price_match.group(1).replace(',', ''))
                        elif isinstance(listing["price"], (int, float)):
                            price = float(listing["price"])
                    except Exception as e:
                        print(f"Warning: Could not parse price from {listing['price']}: {e}")
                    
                    # Create a new apartment record with extracted details
                    new_apartment = Apartment(
                        name=listing["name"],
                        price=price,
                        description=listing["description"],
                        available_date=available_date,
                        days_on_market=days_on_market,
                        link=listing["link"],
                        image_urls=[] # TODO: Add image urls
                    )
                    db_session.add(new_apartment)
                    db_session.flush()
            except IntegrityError as conflict_error:
                db_session.rollback()  # Roll back just this record
                print(f"Warning: Conflict or unique violation for listing {listing}: {conflict_error}")
                continue  # Continue with the next listing
            except Exception as e:
                print(f"Warning: Error processing listing {listing['link'] if 'link' in listing else 'unknown'}: {e}")
                continue  # Continue with the next listing
        
        # Commit the session at the end
        try:
            db_session.commit()
        except Exception as e:
            db_session.rollback()
            print(f"Error committing to database: {e}")

async def save_listings_to_db_from_urls(listings, Session):
    """
    Save apartment listings to the database.
    
    Args:
        listings: List of listing URLs to save
        Session: SQLAlchemy sessionmaker
    
    Returns:
        int: Number of successfully inserted listings
    """
    try:
        # Create a new session for this transaction
        db_session = Session()
        
        # Create a Playwright browser to visit each listing
        async with async_playwright() as playwright:
            browser = await playwright.chromium.launch(headless=True)
            page = await browser.new_page()
            
            successful_inserts = 0
            for listing in listings:
                if listing and isinstance(listing, str):
                    try:
                        # Check if the apartment already exists
                        existing_apartment = db_session.query(Apartment).filter_by(link=listing).first()
                        if not existing_apartment:
                            # Extract apartment details
                            apartment_details = await extract_apartment_details(page, listing)
                            
                            if apartment_details:
                                # Create a new apartment record with extracted details
                                new_apartment = Apartment(
                                    link=listing,
                                    name=apartment_details["name"],
                                    description=apartment_details["description"],
                                    available_date=apartment_details["available_date"],
                                    days_on_market=apartment_details["days_on_market"],
                                    image_urls=apartment_details["image_urls"]
                                )
                                db_session.add(new_apartment)
                                db_session.flush()  # Flush to detect conflicts for this record
                                successful_inserts += 1
                    except IntegrityError as conflict_error:
                        db_session.rollback()  # Roll back just this record
                        print(f"Warning: Conflict or unique violation for listing {listing}: {conflict_error}")
                        continue  # Continue with the next listing
            
            # Close the browser
            await browser.close()
        
        # Commit the changes to the database
        db_session.commit()
        print(f"Successfully stored {successful_inserts} listings in the database")
        return successful_inserts
    except Exception as db_error:
        db_session.rollback()
        print(f"Database error: {db_error}")
        print(f"Database error stack trace:\n{traceback.format_exc()}")
        return 0
    finally:
        # Always close the session
        db_session.close()

# Flexible Playwright Script Generation with LLMs
In order to be resilient to changes in the DOM, we can use LLMs to generate playwright scripts. This adds some complexity but enables the script to work with a wider variety of pages and not break on single DOM changes.

## Evaluator-Optimizer Pattern
This script uses the evaluator-optimizer pattern to generate playwright scripts. The evaluator is a LLM that evaluates the quality of the playwright script and the optimizer is a LLM that optimizes the playwright script.

In [34]:
from playwright.async_api import Page
import asyncio


async def get_text_content_with_timeout(
    page: Page, selector: str, timeout_s: float = 5.0
):
    try:
        locator = page.locator(selector)
        content = await asyncio.wait_for(locator.text_content(), timeout=timeout_s)
        return content.strip() if content else None
    except (asyncio.TimeoutError, Exception):
        return None


async def scrape_listing_details(page: Page) -> dict[str, any]:
    selectors = {
        "name": '[data-testid="homeAddress"]',
        "price": '[data-testid="priceInfo"]',
        "description": '[data-testid="about-section"]',
        "available_date": '[data-testid="rentalListingSpec-available"]',
        "days_on_market": '[data-testid="rentalListingSpec-daysOnMarket"]',
    }

    apartment_data = {key: None for key in selectors.keys()}

    for field, selector in selectors.items():
        apartment_data[field] = await get_text_content_with_timeout(page, selector)

    return apartment_data

In [35]:
# example_apt_scraping_url = "https://streeteasy.com/building/360-east-57-street-new_york/17b?featured=1&utm_campaign=rental_listing&utm_medium=share&utm_source=web&lstt=AZRLueAE5AOrBuTl4tH054yxB913Hsdz7PiVqTB9EAvExTscpksQ1PbOco3MPODNIXE386HUz-gS6KwV"


# async def test_scrape_listing_playwright_script():
#     res = []
#     async with async_playwright() as playwright:
#         browser = await playwright.chromium.launch(headless=False)
#         page = await browser.new_page()
#         exec_globals = {
#             "page": page,
#             "asyncio": asyncio,
#             "playwright": playwright,
#         }
#         await page.goto(example_apt_scraping_url, timeout=120000)
#         dom_content = await page.content()
#         a11y_tree = await page.accessibility.snapshot()
#         error_message = ""
#         listing_details = await scrape_listing_details(page)
#         print(f"listing_details = {listing_details}")
#         res.append(listing_details)
#         await page.close()
#     return res


# res = await test_scrape_listing_playwright_script()
# print(res)

In [36]:
async def main():
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(headless=False)

    for website in config.websites:
        print(f"Navigating to {website}")
        page = await browser.new_page()

        try:
            retry_count = 0 
            error_message = ""
                
            while retry_count < config.script_generation_retries:
                await page.goto(website, timeout=100000)
                dom_content = await page.content()
                print(f"DOM content length: {len(dom_content)} characters")
                a11y_tree = await page.accessibility.snapshot()
                print(
                    f"Accessibility tree obtained with {len(a11y_tree.get('children', []))} root elements"
                )
                print(a11y_tree)

                print("Generating Playwright script...")
                # script = await generate_search_playwright_script(website, dom_content, a11y_tree, error_message)

                # NOTE: Re-enable if testing a playwright script
                script = await get_test_playwright_script()
                print(script)

                # Extract and execute the playwright script
                extracted_script = await extract_playwright_script(script)
                try:
                    # Execute the extracted script against the current page
                    exec_globals = {
                        "page": page,
                        "asyncio": asyncio,
                        "playwright": playwright,
                    }
                    # TODO: Is there a more secure way to do this than exec?
                    # TODO: How to handle dynamic playwright execution?
                    await execute_llm_script(extracted_script, exec_globals)
                    print("Apartment search/filter script completed, starting listing scraping script")
                    # apartment_scraper_script = await generate_save_listings_playwright_script(website, dom_content, a11y_tree, error_message)
                    apartment_scraper_script = await get_test_save_listings_playwright_script()
                    extracted_listings_script = await extract_playwright_script(apartment_scraper_script)
                    print(f"Generated script = {apartment_scraper_script}")
                    listings = await execute_llm_script(extracted_listings_script, exec_globals)
                    print(f"listings = {listings}")

                    listing_details_list: list[dict[str, any]] = []
                    for listing in listings:
                        print(f"scraping listing = {listing}")
                        await page.goto(listing, timeout=60000)
                        dom_content = await page.content()
                        a11y_tree = await page.accessibility.snapshot()
                        # scrape_listing_script = await generate_test_scrape_listing_playwright_script(website, dom_content, a11y_tree, error_message)
                        # extracted_scrape_listing_script = await extract_playwright_script(scrape_listing_script)
                        # print(f"extracted_scrape_listing_script = {extracted_scrape_listing_script}")
                        # listing_details = await execute_llm_script(extracted_scrape_listing_script, exec_globals)
                        listing_details = await scrape_listing_details(page)
                        listing_details["link"] = listing
                        print(f"listing_details = {listing_details}")
                        listing_details_list.append(listing_details)
                        await save_listings_to_db(listing_details_list, Session)

                        # TODO: Implement exponential backoff for politeness
                        await asyncio.sleep(5)
                    
                    break  # Exit the retry loop on success
                except Exception as script_error:
                    retry_count += 1
                    error_message = str(script_error) + "\n" + traceback.format_exc()
                    print(f"Error executing generated script (attempt {retry_count}/{config.script_generation_retries}): {script_error}")
                    print(f"Script execution stack trace:\n{traceback.format_exc()}")
                    if retry_count >= config.script_generation_retries:
                        raise Exception("Maximum retry attempts reached")
                    else:
                        print(f"Retrying in 2 seconds...")
                        await asyncio.sleep(2)

                await page.close()

                # TODO: Implement exponential backoff for politeness when visiting
                await asyncio.sleep(2)

        except Exception as e:
            print(f"Error processing {website}: {e}")
            print(f"Call stack:\n{traceback.format_exc()}")
            await page.close()
        break
    await browser.close()
    await playwright.stop()


await asyncio.gather(main())

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("[data-testid=\\"rentalListingSpec-daysOnMarket\\"]")\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("[data-testid=\"rentalListingSpec-daysOnMarket\"]")



Navigating to https://streeteasy.com/for-rent/nyc
DOM content length: 1230069 characters
Accessibility tree obtained with 391 root elements
{'role': 'WebArea', 'name': 'NYC Apartments for Rent - Updated Daily | StreetEasy', 'children': [{'role': 'link', 'name': 'Skip Navigation'}, {'role': 'link', 'name': 'StreetEasy Logo'}, {'role': 'link', 'name': 'Advertise Link'}, {'role': 'button', 'name': 'Sign In / Register'}, {'role': 'button', 'name': 'Rent', 'haspopup': 'menu'}, {'role': 'button', 'name': 'Buy', 'haspopup': 'menu'}, {'role': 'link', 'name': 'Sell Your Home', 'haspopup': 'menu'}, {'role': 'link', 'name': 'Buildings', 'haspopup': 'menu'}, {'role': 'button', 'name': 'Resources', 'haspopup': 'menu'}, {'role': 'link', 'name': 'Blog', 'haspopup': 'menu'}, {'role': 'text', 'name': 'e.g. address, building, agent'}, {'role': 'textbox', 'name': 'e.g. address, building, agent'}, {'role': 'button', 'name': 'Search'}, {'role': 'heading', 'name': 'LOCATION', 'level': 5}, {'role': 'button',

Future exception was never retrieved
future: <Future finished exception=TimeoutError('Timeout 30000ms exceeded.\nCall log:\n  - waiting for locator("[data-testid=\\"rentalListingSpec-daysOnMarket\\"]")\n')>
playwright._impl._errors.TimeoutError: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("[data-testid=\"rentalListingSpec-daysOnMarket\"]")



listing_details = {'name': '1 University Place #8J', 'price': '$5,450For Rent', 'description': 'About*MULTIPLE APPSExtra Large 800 square foot 1- bedroom converted to 2-bedroom in Pre-war 24 hour Prewar Doorman building across from Washington Square Park.- Cannot tell apartment was originally a one bedroom. Main bedroom can fit a king size bedroom set. Second bedroomcan fit a Queen size bed with full bedroom set.- Landlord will convert back to original 1-bedroom if desired.- Hardwood floors and high 9’0 Beamed Ceilings.show full description', 'available_date': 'Available6/2/2025', 'days_on_market': 'Days on market25 days', 'link': 'https://streeteasy.com/building/1-university-place-new_york/8j?featured=1'}
scraping listing = https://streeteasy.com/building/lincoln-at-bankside/n0847


Future exception was never retrieved
future: <Future finished exception=TimeoutError('Timeout 30000ms exceeded.\nCall log:\n  - waiting for locator("[data-testid=\\"rentalListingSpec-daysOnMarket\\"]")\n')>
playwright._impl._errors.TimeoutError: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("[data-testid=\"rentalListingSpec-daysOnMarket\"]")



listing_details = {'name': '5 Lincoln Avenue #N-0847', 'price': '$5,568For RentPrice Increase$83No fee', 'description': 'AboutNEWLY RELEASED LINCOLN AT BANKSIDE CURRENT SPECIAL OFFERS: + Reduced Security Deposits as low as $99 + Free Amenities + Ask about complimentary parking for 3 months +We will cover your move!* This spacious three-bedroom corner apartment offers the perfect blend of comfort and luxury, featuring two expansive terraces with breathtaking views. The bright, airy layout is designed for modern living, with plenty of room for relaxation and entertaining.show full description', 'available_date': 'AvailableAvailable now', 'days_on_market': 'Days on market91 days', 'link': 'https://streeteasy.com/building/lincoln-at-bankside/n0847'}
scraping listing = https://streeteasy.com/building/maison-78/20b?infeed=1
listing_details = {'name': '356 East 78th Street #20B', 'price': '$17,060For RentNo fee', 'description': 'AboutIntroducing Maison 78, a collection of premier Upper East S

Future exception was never retrieved
future: <Future finished exception=TimeoutError('Timeout 30000ms exceeded.\nCall log:\n  - waiting for locator("[data-testid=\\"rentalListingSpec-daysOnMarket\\"]")\n')>
playwright._impl._errors.TimeoutError: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("[data-testid=\"rentalListingSpec-daysOnMarket\"]")

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("[data-testid=\\"rentalListingSpec-daysOnMarket\\"]")\n')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed
Call log:
  - waiting for locator("[data-testid=\"rentalListingSpec-daysOnMarket\"]")

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed\nCall log:\n  - waiting for locator("[data-testid=\\"rentalListingSpec-daysOnMarket\\"]")

[None]